In [1]:
import polars as pl
import plotly.graph_objects as go
import plotly.io as pio
import mappings
import plotly.express as px

# Set the theme once
pio.templates.default = "plotly_white"

from util import df_prop

In [2]:
df_pre = pl.read_csv("OriginalData/Preprogram_Survey.csv")
N_pre = df_pre.shape[0]

df_pre = df_pre.with_columns(
    pl.col("What is your current education status?").map_elements(lambda x: "True" if ("Currently" in x) else "False", return_dtype=pl.String).alias("College Student"),
    pl.col("What is your gender identity?").map_elements(lambda x: "Female" if ("Female" == x) else "Not Female", return_dtype=pl.String).alias("Gender"),
    pl.lit(1).alias("Count") 
)

df_post = pl.read_csv("./OriginalData/PostOrientation_Survey.csv")
N_post = df_post.shape[0]

df_post = df_post.with_columns(
    pl.col("What is your current education status?").map_elements(lambda x: "True" if ("Currently" in x) else "False", return_dtype=pl.String).alias("College Student"),
    pl.col("What is your gender identity?").map_elements(lambda x: "Female" if ("Female " == x) else "Not Female", return_dtype=pl.String).alias("Gender"),
    pl.lit(1).alias("Count") 
)

In [3]:
post_orientation_mapping = {
    "How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?": [
        "I can manage challenges without giving up"
    ],
    "After learning about emotional intelligence, how are you at recognizing your personal emotional triggers?": [
        "Stressed or overwhelmed"
    ],
    "After the Communication Styles workshop, how well do you understand your own communication style?": [
        "Communicating professionally (verbal and written)"
    ],
    "Are you more or less motivated about setting and actually reaching your goals after the goals workshop?": [
        "Motivated to work toward your goals",
        "Focused and productive"
    ],
    "What is your current clarity level about your possible career paths?": [
        "Do you have a *career plan or clear next steps?*"
    ],
    "How confident are you in your ability *to manage your money* this summer?": [
        "Creating a personal budget",
        "Understanding credit and credit scores",
    ],
    "After the personal branding session, how confident are you in *how you show up professionally*?": [
        "I know how to present myself professionally in different settings"
    ],
    "Overall, how much do you feel you grew during the orientation?": []
}

all_matched_columns = [col for cols in post_orientation_mapping.values() for col in cols]


In [4]:
df_pre_comp = df_pre.select(all_matched_columns).to_pandas()

In [5]:
response_mapping = {
    "I can manage challenges without giving up": {
        'Strongly disagree': 0,
        'Disagree': 1,
        'Neutral ': 2,
        'Agree': 3,
        'Strongly agree': 4,
        None: None
    },
    "Stressed or overwhelmed": {
        'Very Often': 0,
        'Often': 1,
        'Sometimes': 2,
        'Rarely': 3,
        'Never': 4,
        None: None
    },
    "Communicating professionally (verbal and written)": {
        'Not confident': 0,
        'Slightly ': 1,
        'Somewhat': 2,
        'Very': 3,
        'Extremely': 4
    },
    "Motivated to work toward your goals": {
        'Very Often': 4,
        'Often': 3,
        'Sometimes': 2,
        'Rarely': 1,
        'Never': 0,
        None: None
    },
    "Focused and productive": {
        'Very Often': 4,
        'Often': 3,
        'Sometimes': 2,
        'Rarely': 1,
        'Never': 0,
        None: None
    },
    "Do you have a *career plan or clear next steps?*": {
        "I’m still exploring and need guidance": 0,
        'I have a general idea but need help organizing it': 1,
        "Yes, I have a clear plan I’m working on": 2,
        None: None
    },
    "Creating a personal budget": {
        'Not familiar': 0,
        'Slightly': 1,
        'Somewhat': 2,
        'Very': 3,
        'Extremely': 4
    },
    "Understanding credit and credit scores": {
        'Not familiar': 0,
        'Slightly': 1,
        'Somewhat': 2,
        'Very': 3,
        'Extremely': 4
    },
    "I know how to present myself professionally in different settings": {
        'Strongly disagree': 0,
        'Neutral ': 1,
        'Agree': 2,
        'Strongly agree': 3,
        None: None
    }
}


In [6]:
for column, mapping in response_mapping.items():
    if column in df_pre_comp.columns:
        df_pre_comp[column] = df_pre_comp[column].replace(mapping)

/tmp/ipykernel_42381/2928776004.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pre_comp[column] = df_pre_comp[column].replace(mapping)


In [7]:
competency_to_item = {
    'Confidence in Managing Emotions': 'I can manage challenges without giving up',
    'Recognizing Emotional Triggers': 'Stressed or overwhelmed',
    'Understanding Communication Style': 'Communicating professionally (verbal and written)',
    'Motivation for Goal Setting': 'Motivated to work toward your goals',
    'Clarity on Career Paths': 'Do you have a *career plan or clear next steps?*',
    'Confidence in Money Management': 'Creating a personal budget',
    'Confidence in Professional Presence': 'I know how to present myself professionally in different settings'
}

In [8]:
reversed_dict = {v: k for k, v in competency_to_item.items()}

pre_std = pl.DataFrame(df_pre_comp).select(competency_to_item.values()).rename(reversed_dict)

In [9]:
pre_std = pre_std.unpivot(pre_std.columns, variable_name = "Competency", value_name = "value").group_by("Competency").mean().with_columns( pl.lit("Pre-Orientation"))

In [10]:
df_growth = df_post.select(["#"] + mappings.growth_questions).to_pandas()

df_growth = pl.DataFrame(df_growth.replace(mappings.all_post_orientation_entry_mappings).rename(columns=mappings.original_to_concise_question_names))

# Define the logical order for each set of responses
# These lists should contain the *standardized* entries in your desired order
order_confidence_emotions = ['Highly Confident', 'Very Confident', 'Slightly Confident', 'Not Confident At All']
order_emotional_triggers = ['Significantly Better', 'Much Better', 'Slightly Better', 'Not Better At All']
order_communication_style = ['Clearly Understand', 'Understand & Improving', 'General Understanding', 'Do Not Understand']
order_goal_motivation = ['Highly Motivated', 'Much More Motivated', 'Slightly More Motivated', 'No Change', 'Less Motivated']
order_career_paths = ['Very Clear', 'Solid Ideas', 'Slightly Clearer', 'Still Unsure']
order_money_management = ['Highly Confident', 'Very Confident', 'Somewhat Confident', 'Not Confident At All'] # Duplicate of confidence, but good to keep explicit
order_professional_presence = ['Highly Confident', 'Very Confident', 'Somewhat Confident', 'Slightly More Confident', 'Not Yet Confident']
order_overall_growth = ['More Than Expected', 'Significant Growth', 'Some Growth', 'Slight Growth', 'No Growth At All']

# A dictionary to map question names (or a unique identifier for them) to their respective orders
post_orientation_question_orders = {
    "Confidence in Managing Emotions": order_confidence_emotions,
    "Recognizing Emotional Triggers": order_emotional_triggers,
    "Understanding Communication Style": order_communication_style,
    "Motivation for Goal Setting": order_goal_motivation,
    "Clarity on Career Paths": order_career_paths,
    "Confidence in Money Management": order_money_management,
    "Confidence in Professional Presence": order_professional_presence,
    "Overall Growth": order_overall_growth
}



df_numeric = df_growth.drop(["#"]).to_pandas().replace(mappings.all_post_orientation_score_map)

# Step 3: Average across all responses per skill
avg_scores = df_numeric.mean() + 1

/tmp/ipykernel_42381/1716853796.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_numeric = df_growth.drop(["#"]).to_pandas().replace(mappings.all_post_orientation_score_map)


In [11]:
post_std = pl.DataFrame(df_numeric).unpivot(df_numeric.columns, variable_name = "Competency").group_by("Competency").mean().with_columns( pl.lit("Post-Orientation")).filter(pl.col("Competency").is_in(pre_std["Competency"]))

In [12]:
df_comp = pl.concat([pre_std, post_std]).rename({"literal": "When"}).filter(
    pl.col("Competency") != "Confidence in Managing Emotions",
    pl.col("Competency") != "Motivation for Goal Setting"
)

In [13]:
df_comp = df_comp.with_columns(pl.col("value") + 1).with_columns( pl.col("Competency").replace({"Recognizing Emotional Triggers": "Emotional Management"}))

In [14]:
# Create grouped bar chart
fig = px.bar(
    df_comp,
    x="Competency",
    y="value",
    color="When",
    barmode="group",
    labels={"value": "Average Score", "Competency": "Competency"},
    title="Pre- vs Post-Orientation Scores by Competency"
)

fig.update_layout(xaxis_tickangle=-45, yaxis_range = [0,5])
fig.show()

/usr/lib/python3/dist-packages/plotly/express/_core.py:1992: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [17]:
for c in df_pre.select(list(competency_to_item.values())).columns:
    df_prop(df_pre, c, N_pre)

I can manage challenges without giving up
shape: (6, 3)
┌─────────────────────────────────┬─────┬──────┐
│ I can manage challenges withou… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ null                            ┆ 1   ┆ 0.01 │
│ Disagree                        ┆ 1   ┆ 0.01 │
│ Strongly agree                  ┆ 31  ┆ 0.31 │
│ Neutral                         ┆ 20  ┆ 0.2  │
│ Agree                           ┆ 45  ┆ 0.45 │
│ Strongly disagree               ┆ 2   ┆ 0.02 │
└─────────────────────────────────┴─────┴──────┘
Stressed or overwhelmed
shape: (6, 3)
┌─────────────────────────┬─────┬──────┐
│ Stressed or overwhelmed ┆ len ┆ prop │
│ ---                     ┆ --- ┆ ---  │
│ str                     ┆ u32 ┆ f64  │
╞═════════════════════════╪═════╪══════╡
│ Very Often              ┆ 16  ┆ 0.16 │
│ Never                   ┆ 2   ┆ 0.02 │
│ Rarely                  ┆ 13 

In [ ]:
for c in pl.DataFrame(df_growth).select(list(competency_to_item.keys())):
    if 
    df_prop(df_growth, c, N_post)

shape: (85,)
Series: 'Confidence in Managing Emotions' [str]
[
	"Highly Confident"
	"Slightly Confident"
	"Slightly Confident"
	"Highly Confident"
	"Highly Confident"
	…
	"Very Confident"
	"Very Confident"
	"Very Confident"
	"Highly Confident"
	"Highly Confident"
]
shape: (4, 3)
┌─────────────────────────────────┬─────┬──────────┐
│ Confidence in Managing Emotion… ┆ len ┆ prop     │
│ ---                             ┆ --- ┆ ---      │
│ str                             ┆ u32 ┆ f64      │
╞═════════════════════════════════╪═════╪══════════╡
│ Very Confident                  ┆ 45  ┆ 0.529412 │
│ Not Confident At All            ┆ 1   ┆ 0.011765 │
│ Slightly Confident              ┆ 10  ┆ 0.117647 │
│ Highly Confident                ┆ 29  ┆ 0.341176 │
└─────────────────────────────────┴─────┴──────────┘
shape: (85,)
Series: 'Recognizing Emotional Triggers' [str]
[
	"Significantly Better"
	"Much Better"
	"Much Better"
	"Significantly Better"
	"Significantly Better"
	…
	"Much Better"
	"Much B

In [51]:
career_path_mapping = {'I feel very clear about my options and direction ': "Very Clear of My Direction",
 'I have a little more clarity now ': "Have a General Idea",
 'I have solid ideas ': "Very Clear of My Direction",
 'I still feel unsure ': "Unsure of My Direction",
 'I have a general idea but need help organizing it': "Have a General Idea",
 'I’m still exploring and need guidance':"Unsure of My Direction",
 'Yes, I have a clear plan I’m working on': "Very Clear of My Direction"}

df_cp_pre = df_pre.select("Do you have a *career plan or clear next steps?*").with_columns(
    pl.col("Do you have a *career plan or clear next steps?*").replace(career_path_mapping).alias("Clarity of Career Path"),
    pl.lit("Pre-Orientation").alias("When"),
    pl.lit(N_pre).alias("N")
).drop("Do you have a *career plan or clear next steps?*")

In [52]:
df_cp_post = df_post.select('What is your current clarity level about your possible career paths?').with_columns(
    pl.col('What is your current clarity level about your possible career paths?').replace(career_path_mapping).alias("Clarity of Career Path"),
    pl.lit("Post-Orientation").alias("When"),
    pl.lit(N_post).alias("N")
).drop('What is your current clarity level about your possible career paths?')

In [53]:
df_cp = pl.concat([df_cp_pre, df_cp_post])

In [58]:
df_cp.group_by(df_cp.columns).len().drop_nulls().with_columns( (pl.col("len")/pl.col("N")).alias("Proportion")).sort(by = "Clarity of Career Path")

Clarity of Career Path,When,N,len,Proportion
str,str,i32,u32,f64
"""Have a General Idea""","""Pre-Orientation""",100,44,0.44
"""Have a General Idea""","""Post-Orientation""",85,12,0.141176
"""Unsure of My Direction""","""Post-Orientation""",85,5,0.058824
"""Unsure of My Direction""","""Pre-Orientation""",100,19,0.19
"""Very Clear of My Direction""","""Pre-Orientation""",100,36,0.36
"""Very Clear of My Direction""","""Post-Orientation""",85,68,0.8
